# Error mitigation on IBMQ backends with Qiskit


In [2]:
import qiskit
from qiskit_aer import QasmSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

from mitiq import zne
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise

## Setup: Defining a circuit


In [39]:
num_qubits = 4  # 9, 16, 25, 36, 49 (available through benchpress)
# qasm_path = f"square-heisenberg-{num_qubits}.qasm"
qasm_path = f"qft-{num_qubits}.qasm"
circuit = qiskit.qasm2.load(qasm_path)
# circuit.measure_all()

print(circuit)
print(len(circuit))

                                                                     »
q_0: ────────────────────────────────────────────────────────────────»
                                                                     »
q_1: ────────────────────────────────────────────────────────────────»
                                    ┌───┐┌──────────┐┌───┐┌─────────┐»
q_2: ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├»
     ┌─────────┐┌───────┐┌─────────┐└─┬─┘└──────────┘└─┬─┘├─────────┤»
q_3: ┤ Ry(π/2) ├┤ Rx(π) ├┤ Rz(π/4) ├──■────────────────■──┤ Rz(π/8) ├»
     └─────────┘└───────┘└─────────┘                      └─────────┘»
«                                                                  ┌───┐»
«q_0: ─────────────────────────────────────────────────────────────┤ X ├»
«                ┌───┐┌──────────┐           ┌───┐┌─────────┐ ┌───┐└─┬─┘»
«q_1: ───────────┤ X ├┤ Rz(-π/8) ├───────────┤ X ├┤ Rz(π/8) ├─┤ X ├──┼──»
«     ┌─────────┐└─┬─┘└┬───────┬─┘┌─────────┐└─┬─┘└─────────┘ └─┬

We will use the probability of the ground state as our observable to mitigate, the expectation value of which should
evaluate to one in the noiseless setting.

## High-level usage

To use Mitiq with just a few lines of code, we simply need to define a function which inputs a circuit and outputs
the expectation value to mitigate. This function will:

1. [Optionally] Add measurement(s) to the circuit.
2. Run the circuit.
3. Convert from raw measurement statistics (or a different output format) to an expectation value.

We define this function in the following code block. Because we are using IBMQ backends, we first load our account.


**Note:** Using an IBM quantum computer requires a valid IBMQ account. See <https://quantum-computing.ibm.com/>
for instructions to create an account, save credentials, and see online quantum computers.


In [35]:
from qiskit_aer.noise import NoiseModel


# HARDWARE
service = QiskitRuntimeService()
hard_backend = service.least_busy(operational=True, simulator=False)
print(hard_backend)


# SIMULATOR
noise_model = NoiseModel.from_backend(hard_backend)
sim_backend = QasmSimulator(method="density_matrix", noise_model=noise_model)

sim_backend_ideal = QasmSimulator()

<IBMBackend('ibm_kyiv')>


In [45]:
def ibmq_executor(circuit: qiskit.QuantumCircuit, backend, shots: int = 1024) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run (assumes no measurements).
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        basis_gates=None,
        optimization_level=0,  # Important to preserve folded gates.
    )
    exec_circuit.measure_active()

    num_qubits = circuit.num_qubits
    print("num_qubits", num_qubits)
    # Run the circuit
    sampler = Sampler(mode=backend)
    job = sampler.run([exec_circuit], shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result()[0].data.measure.get_counts()
    print(counts)
    ground_state = "0" * num_qubits
    expectation_value = counts.get(ground_state, 0.0) / shots
    return expectation_value


def qasm_simulator_executor(circuit: qiskit.QuantumCircuit, shots: int = 1024) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run.
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=QasmSimulator(),
        basis_gates=None,
        optimization_level=0,  # Important to preserve folded gates.
    )
    exec_circuit.measure_active()

    num_qubits = exec_circuit.num_qubits
    # Run the circuit
    sampler = Sampler(mode=QasmSimulator())
    job = sampler.run([exec_circuit], shots=shots)

    # Convert from raw measurement counts to the expectation value
    # counts = job.result().get_counts()
    counts = job.result()[0].data.measure.get_counts()
    ground_state = "0" * num_qubits
    expectation_value = counts.get(ground_state, 0.0) / shots
    return expectation_value

In [46]:
ideal = ibmq_executor(circuit, sim_backend_ideal)
noisy = ibmq_executor(circuit, sim_backend)
noisy_hardware = ibmq_executor(circuit, hard_backend)

print("ideal:", ideal)
print("noisy:", noisy)
print("hardware:", noisy_hardware)

num_qubits 4
{'1100': 81, '1000': 59, '1111': 63, '1010': 65, '0100': 62, '0000': 72, '0111': 71, '0011': 60, '1001': 76, '0110': 55, '0101': 63, '1101': 61, '0001': 52, '1011': 58, '0010': 58, '1110': 68}
num_qubits 4
{'1011': 77, '1111': 68, '1110': 73, '1000': 61, '1101': 73, '0010': 54, '1100': 63, '0000': 58, '0100': 79, '0110': 47, '0101': 55, '1001': 63, '0011': 59, '0001': 74, '0111': 59, '1010': 61}
num_qubits 4
{'1011': 65, '0001': 53, '1000': 58, '1111': 74, '1100': 76, '1110': 56, '0101': 102, '0100': 110, '0111': 49, '1101': 80, '0110': 47, '0010': 45, '1001': 63, '0000': 60, '1010': 48, '0011': 38}
ideal: 0.0703125
noisy: 0.056640625
hardware: 0.05859375


In [17]:
backend

<IBMBackend('ibm_kyiv')>

At this point, the circuit can be executed to return a mitigated expectation value by running {func}`zne.execute_with_zne`,
as follows.


In [58]:
from mitiq import lre
from functools import partial

hardware_executor = partial(ibmq_executor, backend=hard_backend)

unmitigated = ibmq_executor(circuit, hard_backend)
mitigated = lre.execute_with_lre(
    circuit, hardware_executor, degree=3, fold_multiplier=2, num_chunks=5
)
print(f"Unmitigated result {unmitigated:.3f}")
print(f"Mitigated result {mitigated:.3f}")

num_qubits 4
{'0101': 81, '0110': 115, '0001': 59, '1001': 58, '1101': 83, '0111': 30, '0011': 58, '1010': 51, '0010': 88, '0100': 88, '1110': 45, '1000': 63, '1011': 45, '1100': 63, '1111': 42, '0000': 55}
num_qubits 4
{'1001': 61, '0010': 109, '0000': 63, '0011': 49, '1000': 52, '0110': 104, '1100': 65, '1010': 47, '0101': 78, '0100': 80, '1111': 38, '1011': 52, '1101': 57, '0111': 60, '1110': 54, '0001': 55}
num_qubits 4
{'0100': 98, '0010': 83, '0000': 108, '0001': 53, '1101': 42, '1000': 70, '1010': 86, '0110': 96, '1111': 37, '1100': 72, '0101': 63, '1110': 56, '1011': 48, '0011': 34, '1001': 42, '0111': 36}
num_qubits 4
{'0101': 51, '0001': 53, '0000': 104, '0111': 59, '1010': 56, '0110': 79, '0010': 124, '1110': 45, '1011': 45, '1100': 69, '1000': 93, '0100': 89, '1101': 38, '1001': 26, '1111': 28, '0011': 65}
num_qubits 4
{'0001': 47, '0000': 78, '1001': 47, '0010': 63, '1100': 63, '0111': 62, '1110': 92, '1011': 29, '1101': 41, '1000': 63, '0110': 84, '0101': 92, '1111': 38, 

In [59]:
print(f"ideal result {ibmq_executor(circuit, sim_backend_ideal)}")

num_qubits 4
{'0111': 58, '0101': 60, '1110': 72, '1010': 63, '1000': 59, '1101': 62, '0100': 68, '0001': 49, '1011': 75, '1111': 73, '0000': 69, '0110': 47, '1100': 83, '0011': 59, '0010': 66, '1001': 61}
ideal result 0.0673828125


As long as a circuit and a function for executing the circuit are defined, the {func}`zne.execute_with_zne` function can
be called as above to return zero-noise extrapolated expectation value(s).

## Options

Different options for noise scaling and extrapolation can be passed into the {func}`zne.execute_with_zne` function.
By default, noise is scaled by locally folding gates at random, and the default extrapolation is Richardson.

To specify a different extrapolation technique, we can pass a different {class}`.Factory` object to {func}`zne.execute_with_zne`. The
following code block shows an example of using linear extrapolation with five different (noise) scale factors.


In [6]:
linear_factory = zne.inference.LinearFactory(scale_factors=[1.0, 1.5, 2.0, 2.5, 3.0])
mitigated = zne.execute_with_zne(circuit, ibmq_executor, factory=linear_factory)
print(f"Mitigated result {mitigated:.3f}")

Mitigated result 0.977


To specify a different noise scaling method, we can pass a different function for the argument `scale_noise`. This
function should input a circuit and scale factor and return a circuit. The following code block shows an example of
scaling noise by global folding (instead of local folding, the default behavior for
{func}`zne.execute_with_zne`).


In [7]:
mitigated = zne.execute_with_zne(
    circuit, ibmq_executor, scale_noise=zne.scaling.fold_global
)
print(f"Mitigated result {mitigated:.3f}")

Mitigated result 0.962


Any different combination of noise scaling and extrapolation technique can be passed as arguments to
{func}`zne.execute_with_zne`.

## Lower-level usage

Here, we give more detailed usage of the Mitiq library which mimics what happens in the call to
{func}`zne.execute_with_zne` in the previous example. In addition to showing more of the Mitiq library, this
example explains the code in the previous section in more detail.

First, we define factors to scale the circuit length by and fold the circuit using the `fold_gates_at_random`
local folding method.


In [8]:
scale_factors = [1.0, 1.5, 2.0, 2.5, 3.0]
folded_circuits = [
    zne.scaling.fold_gates_at_random(circuit, scale) for scale in scale_factors
]

# Check that the circuit depth is (approximately) scaled as expected
for j, c in enumerate(folded_circuits):
    print(
        f"Number of gates of folded circuit {j} scaled by: {len(c) / len(circuit):.3f}"
    )

Number of gates of folded circuit 0 scaled by: 1.000
Number of gates of folded circuit 1 scaled by: 1.364
Number of gates of folded circuit 2 scaled by: 1.909
Number of gates of folded circuit 3 scaled by: 2.273
Number of gates of folded circuit 4 scaled by: 2.818


For a noiseless simulation, the expectation of this observable should be 1.0 because our circuit compiles to the identity.
For a noisy simulation, the value will be smaller than one. Because folding introduces more gates and thus more noise,
the expectation value will decrease as the length (scale factor) of the folded circuits increase. By fitting this to
a curve, we can extrapolate to the zero-noise limit and obtain a better estimate.

Below we execute the folded circuits using the `backend` defined at the start of this example.


In [9]:
shots = 8192

# Transpile the circuit so it can be properly run
exec_circuit = qiskit.transpile(
    folded_circuits,
    backend=backend,
    basis_gates=noise_model.basis_gates if noise_model else None,
    optimization_level=0,  # Important to preserve folded gates.
)

# Run the circuit
job = backend.run(exec_circuit, shots=shots)

**Note:** We set the `optimization_level=0` to prevent any compilation by Qiskit transpilers.

Once the job has finished executing, we can convert the raw measurement statistics to observable values by running the
following code block.


In [10]:
all_counts = [job.result().get_counts(i) for i in range(len(folded_circuits))]
expectation_values = [counts.get("0") / shots for counts in all_counts]
print(f"Expectation values:\n{expectation_values}")

Expectation values:
[0.9044189453125, 0.877197265625, 0.8328857421875, 0.810546875, 0.76708984375]


We can now see the unmitigated observable value by printing the first element of `expectation_values`. (This value
corresponds to a circuit with scale factor one, i.e., the original circuit.)


In [11]:
print("Unmitigated expectation value:", round(expectation_values[0], 3))

Unmitigated expectation value: 0.904


Now we can use the static `extrapolate` method of {class}`zne.inference.Factory` objects to extrapolate to the zero-noise limit. Below we use an exponential fit and print out the extrapolated zero-noise value.


In [12]:
zero_noise_value = zne.ExpFactory.extrapolate(
    scale_factors, expectation_values, asymptote=0.5
)
print(f"Extrapolated zero-noise value:", round(zero_noise_value, 3))

Extrapolated zero-noise value: 1.002
